#### DLS Project - Recommendation System using GNN
1. Unipartite Model
2. Script to load the model output
3. Find sample cases for manual review of recommendations made by the model
4. Input Files availabe at - https://indiana-my.sharepoint.com/:f:/g/personal/abbajpai_iu_edu/EhlGzk26Td9EvsykPVD6kcABji4dRHR2fvRf6E8d1FdlKA?e=fusaup

In [1]:
import pandas as pd
import numpy as np
import os
PATH = '/N/project/APRS/model_user_results'

##### Loading the saved embeddings and dataframe

In [2]:
# load the dataframe
data_df_3_sample = pd.read_pickle(os.path.join(PATH, 'data_df_3_sample.pkl'))

In [3]:
# load the embeddings
h = pd.read_csv(os.path.join(PATH,'embedding_vector_user.csv'))

In [4]:
# load the node list
nodes_list = pd.read_pickle(os.path.join(PATH,'node_list.pkl'))
nodes_list = nodes_list[0].tolist()

##### Generating results

In [72]:
# Step 1: Select any category
data_df_3_sample.categoryLast.unique()

array([' Thrillers & Suspense', ' Americas',
       ' Research &amp; Publishing Guides', ' United States',
       ' Thrillers &amp; Suspense', ' Thriller & Suspense',
       ' Genre Fiction', ' British & Irish', ' British &amp; Irish',
       ' Action &amp; Adventure', ' Fantasy', ' Science Fiction',
       ' Crafts & Hobbies', ' Historical', ' Contemporary',
       ' Arts &amp; Literature', ' Mystery', ' Literature &amp; Fiction',
       ' Politics &amp; Government', ' Military', '',
       ' Science Fiction &amp; Fantasy', ' Time Travel',
       ' Holidays &amp; Celebrations', ' Paranormal', ' Graphic Novels',
       ' Short Stories &amp; Anthologies',
       ' Growing Up &amp; Facts of Life', ' "Womens Fiction"',
       ' Humor & Satire', ' Bible Study &amp; Reference',
       ' Literature & Fiction', ' Religion &amp; Spirituality',
       ' Science Fiction & Fantasy', ' Photography &amp; Video', ' Humor',
       ' Professionals & Academics', ' "Childrens Books"',
       ' Pets &amp

In [20]:
# Step 2: Select sample examples >> Beaware of space in the category
sample_users = data_df_3_sample[data_df_3_sample["categoryLast"].isin([' Science Fiction'])]

In [29]:
# Step 3: Pick an example from this table
sample_users.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,...,imageURL,imageURLHighRes,details,categoryLast,asin_right,buy_frequency,user_id_idx,item_id_idx,price_right,price_category
14,3.0,False,"12 27, 2017",A21FZK6X6CVS2Q,0999214756,JMScott,The majority of Mr. McGinnis' books revolve ar...,Young Adult Science Fiction,1514332800,{'Format:': ' Kindle Edition'},...,[],[],NaN,Science Fiction,143,1,129105,147463,21.99,0
15,5.0,False,"12 1, 2017",A21FZK6X6CVS2Q,1502544296,JMScott,Commander Creed creates a new type of human or...,The final confrontation with the Purple Emperors,1512086400,{'Format:': ' Kindle Edition'},...,[],[],NaN,Science Fiction,143,1,129105,194416,21.99,0
17,5.0,False,"06 5, 2016",A21FZK6X6CVS2Q,1519008570,JMScott,Read the whole trilogy from start to finish......,Ended the Trilogy with all the questions answe...,1465084800,{'Format:': ' Kindle Edition'},...,[],[],NaN,Science Fiction,143,1,129105,206478,21.99,0
20,5.0,False,"01 12, 2018",A21FZK6X6CVS2Q,0989931994,JMScott,When Earth receives a psychic signal in the 19...,An alien signal warning Earth of a coming inva...,1515715200,{'Format:': ' Kindle Edition'},...,[],[],NaN,Science Fiction,143,1,129105,139083,21.99,0
21,4.0,False,"10 20, 2016",A21FZK6X6CVS2Q,1539316653,JMScott,A former Marine stumbles upon a group of alien...,An ancient alien menace rises awakes on Earth,1476921600,{'Format:': ' Kindle Edition'},...,[],[],NaN,Science Fiction,143,1,129105,220628,21.99,0


In [23]:
# Step 4: Load a reviewer name
reviwerID_val = 'A1PZU88ONVYUHB' # Example >>>>>  CHANGE THIS <<<<<<<<<<<< A2TS60D6XB4KBY
index = [i for i, x in enumerate(nodes_list) if reviwerID_val==x]
print(f'reviewerID index in graph {index}')

reviewerID index in graph [12]


In [33]:
# review the selected reviewer's book id
data_df_3_sample[data_df_3_sample.reviewerID == 'A2ASNCPAZWNTF0']

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,...,imageURL,imageURLHighRes,details,categoryLast,asin_right,buy_frequency,user_id_idx,item_id_idx,price_right,price_category
0,4.0,True,"05 7, 2017",A2ASNCPAZWNTF0,1503936805,Amazon Customer,"Good read but clearly fiction. Moves fast, har...",Great book for military buffs,1494115200,{'Format:': ' Kindle Edition'},...,[],[],NaN,Thrillers & Suspense,11,1,161512,195864,13.5,0
1,5.0,True,"07 12, 2017",A2ASNCPAZWNTF0,1517153158,Amazon Customer,Interesting book with a time travel theme alth...,A real Technothriller,1499817600,{'Format:': ' Kindle Edition'},...,[],[],NaN,Thrillers & Suspense,11,1,161512,204576,13.5,0


In [24]:
# Step 5: Generate the Top k neighbors 
h = np.array(h)
k = 5 # of similar users >>>>>  CHANGE THIS <<<<<<<<<<<<
distances = np.linalg.norm(h - h[index], axis = 1)
# select indices of vectors having the lowest distances from the X
neighbours = np.argpartition(distances, range(0, k))[:k]
node_names = [nodes_list[i] for i in neighbours]
print(f'Top {k} neighbours of {reviwerID_val} :{node_names}')

Top 5 neighbours of A1PZU88ONVYUHB :['A1PZU88ONVYUHB', 'A85FMKX6NA9R5', 'ADIGLF13KB5B5', 'A2I4KH6YJAAOUN', 'AHPV36THBG3SY']


In [25]:
# Step 6 : Table with all similar users and their books 
similar_users_df = data_df_3_sample.loc[data_df_3_sample['reviewerID'].isin(node_names)] # all books are 4 or 5 ratings
#print(similar_users_df['asin'])

In [26]:
# Asin are the suggestions and CategoryLast is its book category
# A2TS60D6XB4KBY is my reviewer and other are closest users
# A2TS60D6XB4KBY bought books on Women's Fiction, Contemprory, Paranormal
# We find lot of suggestion in same category from other users
recommendation = similar_users_df.groupby(['reviewerID','asin','categoryLast'])['overall'].count().reset_index(drop=False)

In [27]:
recommendation #<<< check the complete table

,reviewerID,asin,categoryLast,overall
0,A1PZU88ONVYUHB,0983797803,Genre Fiction,1
1,A1PZU88ONVYUHB,1442389982,Politics &amp; Government,1
2,A1PZU88ONVYUHB,1505414520,Genre Fiction,1
3,A1PZU88ONVYUHB,1505537908,Science Fiction,1
4,A1PZU88ONVYUHB,1507814046,Science Fiction,1
5,A1PZU88ONVYUHB,1512223018,Science Fiction,1
6,A1PZU88ONVYUHB,1530753228,Mystery,1
7,A1PZU88ONVYUHB,1539951464,Mystery,1
8,A1PZU88ONVYUHB,1542651999,Mystery,1
9,A1PZU88ONVYUHB,1935070126,Thrillers &amp; Suspense,1
